In [2]:
import pandas as pd
import distinctipy
import configs.itol_text as itol_text
import os

def get_color_dict_and_info_list(df, col):
    df = df.fillna("None")

    unique_values = df[col].unique()

    color_set = [distinctipy.get_hex(x) for x in distinctipy.get_colors(len(unique_values))]

    color_dict = {value: color_set[i % len(color_set)] for i, value in enumerate(unique_values)}

    info_list = [(df.at[i, "info"], df.at[i, col]) for i in df.index]

    return color_dict, info_list


def generate_itol_colorstrip(col, color_dict, info_list, output_filename):
    with open(output_filename, "w") as f:
        f.write(itol_text.dataset_colorstrip_text.replace("<custom_dataset_label>", col))
        for info, label in info_list:
            f.write(f"{info},{color_dict[label]},{label} \n")

    print(f"File '{output_filename}' has been created.")


def generate_itol_ranges(col, color_dict, info_list, output_filename):
    with open(output_filename, "w") as f:
        f.write(itol_text.dataset_ranges_text.replace("<custom_dataset_label>", col))
        for info, label in info_list:
            f.write(
                f"{info},{info},{color_dict[label]},{color_dict[label]},{color_dict[label]},dashed,2,{label},black,italic\n"
            )

    print(f"File '{output_filename}' has been created.")


def generate_dataset_style(col, color_dict, info_list, output_filename):
    with open(output_filename, "w") as f:
        f.write(itol_text.dataset_style_text.replace("<custom_dataset_label>", col))
        for info, label in info_list:
            if label in color_dict:
                f.write(f"{info},branch,node,{color_dict[label]},1,normal\n")

    print(f"File '{output_filename}' has been created.")


def generate_shape_style(col, colour, info_dict, output_filename):
    with open(output_filename, "w") as f:
        text_to_write = itol_text.shape_text.replace("<custom_dataset_label>", col)
        text_to_write = text_to_write.replace("<col>", col)
        f.write(text_to_write.replace("<custom_color>", colour))

        for info, val in info_dict.items():
            f.write(f"{info},1\n")

    print(f"File '{output_filename}' has been created.")


if __name__ == "__main__":
    input_csv = "input.csv"
    output_dir = "./"

    df = pd.read_csv(input_csv)

    # Example annotation columns
    annotation_cols = ["col1", "col2"]

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Process annotation columns
    for col in annotation_cols:
        if col != 'info' and col in df:
            color_dict, info_list = get_color_dict_and_info_list(df, col)

            generate_itol_colorstrip(
                col, color_dict, info_list, os.path.join(output_dir, f"{col}_itol_colorstrip.txt")
            )

            generate_itol_ranges(
                col, color_dict, info_list, os.path.join(output_dir, f"{col}_itol_ranges.txt")
            )

    # Process single colour annotation columns
    single_colours = [
        distinctipy.get_hex(x) for x in distinctipy.get_colors(len(single_colour_annotation_cols), pastel_factor=0.7)
    ]
    for idx, single_col in enumerate(single_colour_annotation_cols):
        if single_col in df.columns:
            info_dict = get_single_info_dict(df, single_col)

            generate_shape_style(
                single_col, single_colours[idx], info_dict, os.path.join(output_dir, f"{single_col}_itol_dataset_style.txt")
            )


FileNotFoundError: [Errno 2] No such file or directory: 'input.csv'